In [ ]:
import backtest_lib as btl
import numpy as np
from polars import read_csv
from pathlib import Path
from importlib.resources import files
from backtest_lib.engine.decision import (
    target_weights,
    trade,
    combine,
    hold,
    target_holdings,
)
from backtest_lib.portfolio import cash, uniform_portfolio
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    stream=sys.stdout,
    format="[%(levelname)s] %(name)s: %(message)s",
    force=True,
)


def strategy(universe, market, current_portfolio, ctx):
    return target_weights({"EURUSD": 0.8, "GBPUSD": 0.2}) + (
        combine(
            trade("sell", 10, "GBPUSD"),
            trade("buy", 1, "EURUSD"),
        )
    )


def strategy2(universe, market, current_portfolio, ctx):
    return hold()


def strategy3(universe, market, current_portfolio, ctx):
    return target_weights({"EURUSD": 0.8, "GBPUSD": 0.2})


def strategy4(universe, market, current_portfolio, ctx):
    return target_holdings({"EURUSD": 0.8, "GBPUSD": 0.2}, fill_cash=True)


def strategy5(universe, market, current_portfolio, ctx):
    return target_holdings({"EURUSD": 1000, "GBPUSD": 1000}, fill_cash=True)


market = btl.MarketView(
    read_csv(Path(files(btl)).parent / ".." / "docs/assets/data/spot_prices.csv")
)
universe = market.securities

initial_portfolio = cash(1000000)

ann_returns = []

for strategy in (strategy, strategy2, strategy3, strategy4, strategy5):
    bt = btl.Backtest(
        market_view=market, strategy=strategy, initial_portfolio=initial_portfolio
    )
    results = bt.run()
    ann_returns.append(results.annualized_return)

print(ann_returns)

assert all(
    np.isclose(actual, expected)
    for actual, expected in zip(
        ann_returns,
        [
            4.6135321938312046e-05,
            0.0,
            -0.04613709372097763,
            -5.1307766701924606e-08,
            -7.508319201277036e-05,
        ],
    )
)

In [ ]:
market.securities